In [7]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('Tegar', 'Polisi', 2500),
        ('Mayva', 'Musisi', 3000),
        ('Ardhina', 'Pengacara', 5100),
        ('Kezia', 'Dokter', 3000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       Tegar|    Polisi|  2500|
|       Mayva|    Musisi|  3000|
|     Ardhina| Pengacara|  5100|
|       Kezia|    Dokter|  3000|
+------------+----------+------+



In [10]:

df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 3000).show()
df.groupBy("Department").avg("Salary").show()


+------------+------+
|EmployeeName|Salary|
+------------+------+
|       Tegar|  2500|
|       Mayva|  3000|
|     Ardhina|  5100|
|       Kezia|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Ardhina| Pengacara|  5100|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|    Polisi|     2500.0|
|    Musisi|     3000.0|
| Pengacara|     5100.0|
|    Dokter|     3000.0|
+----------+-----------+



In [11]:
df.withColumn('SalaryBonus', df['Salary'] * 0.1).show()
df.withColumn('TotalCompensation', df['Salary'] + df['SalaryBonus']).show()

+------------+----------+------+-----------+
|EmployeeName|Department|Salary|SalaryBonus|
+------------+----------+------+-----------+
|       Tegar|    Polisi|  2500|      250.0|
|       Mayva|    Musisi|  3000|      300.0|
|     Ardhina| Pengacara|  5100|      510.0|
|       Kezia|    Dokter|  3000|      300.0|
+------------+----------+------+-----------+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `SalaryBonus` cannot be resolved. Did you mean one of the following? [`EmployeeName`, `Department`, `Salary`]. SQLSTATE: 42703

In [12]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Kezia|    Dokter|  3000|   1|
|       Mayva|    Musisi|  3000|   1|
|     Ardhina| Pengacara|  5100|   1|
|       Tegar|    Polisi|  2500|   1|
+------------+----------+------+----+



In [25]:
#Load dataset
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("NetflixDataset").getOrCreate()

df = spark.read.csv("/home/zurich/netflix_titles.csv", header=True, inferSchema=True)
df.show(5)
df.printSchema()


+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         NULL|Septem

In [26]:
#Transformasi Dasar
from pyspark.sql import functions as F

df.select("title", "country").show(10)
df.filter(df["release_year"] > 2015).select("title", "release_year").show(10)
df.groupBy("type").count().show()


+--------------------+--------------------+
|               title|             country|
+--------------------+--------------------+
|Dick Johnson Is Dead|       United States|
|       Blood & Water|        South Africa|
|           Ganglands|                NULL|
|Jailbirds New Orl...|                NULL|
|        Kota Factory|               India|
|       Midnight Mass|                NULL|
|My Little Pony: A...|                NULL|
|             Sankofa|United States, Gh...|
|The Great British...|      United Kingdom|
|        The Starling|       United States|
+--------------------+--------------------+
only showing top 10 rows
+--------------------+------------+
|               title|release_year|
+--------------------+------------+
|Dick Johnson Is Dead|        2020|
|       Blood & Water|        2021|
|           Ganglands|        2021|
|Jailbirds New Orl...|        2021|
|        Kota Factory|        2021|
|       Midnight Mass|        2021|
|My Little Pony: A...|        2021|

In [27]:
#Bekerja Dengan Kolom Tambahan
df = df.withColumn("is_recent", F.when(df["release_year"] >= 2020, True).otherwise(False))
df.select("title", "release_year", "is_recent").show(10)
df = df.withColumn("title_length", F.length("title"))
df.select("title", "title_length").show(10)

+--------------------+------------+---------+
|               title|release_year|is_recent|
+--------------------+------------+---------+
|Dick Johnson Is Dead|        2020|     true|
|       Blood & Water|        2021|     true|
|           Ganglands|        2021|     true|
|Jailbirds New Orl...|        2021|     true|
|        Kota Factory|        2021|     true|
|       Midnight Mass|        2021|     true|
|My Little Pony: A...|        2021|     true|
|             Sankofa|        1993|    false|
|The Great British...|        2021|     true|
|        The Starling|        2021|     true|
+--------------------+------------+---------+
only showing top 10 rows
+--------------------+------------+
|               title|title_length|
+--------------------+------------+
|Dick Johnson Is Dead|          20|
|       Blood & Water|          13|
|           Ganglands|           9|
|Jailbirds New Orl...|          21|
|        Kota Factory|          12|
|       Midnight Mass|          13|
|My Lit

In [23]:
#Agregasi & Statistik
df.groupBy("rating").count().orderBy(F.desc("count")).show()
df.groupBy("country").count().orderBy(F.desc("count")).show(10)
df.groupBy("type").agg(F.avg("title_length").alias("avg_title_length")).show()


+-----------------+-----+
|           rating|count|
+-----------------+-----+
|            TV-MA| 3195|
|            TV-14| 2158|
|            TV-PG|  862|
|                R|  796|
|            PG-13|  489|
|            TV-Y7|  334|
|             TV-Y|  307|
|               PG|  286|
|             TV-G|  220|
|               NR|   80|
|                G|   41|
|             NULL|    6|
|         TV-Y7-FV|    6|
|               UR|    3|
|            NC-17|    3|
|             2021|    2|
| November 1, 2020|    1|
| Shavidee Trotter|    1|
|    Adriane Lenox|    1|
|    Maury Chaykin|    1|
+-----------------+-----+
only showing top 20 rows
+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 2805|
|         India|  972|
|          NULL|  832|
|United Kingdom|  419|
|         Japan|  245|
|   South Korea|  199|
|        Canada|  181|
|         Spain|  145|
|        France|  123|
|        Mexico|  110|
+--------------+-----+
only showing top 10 rows
+---

In [28]:
#Window Function
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("type").orderBy(F.desc("release_year"))

df.withColumn("Rank", F.rank().over(windowSpec)) \
  .select("title", "type", "release_year", "Rank") \
  .filter("Rank <= 5") \
  .show(20, truncate=False)


+----------------------------------------------------+-------+-----------------+----+
|title                                               |type   |release_year     |Rank|
+----------------------------------------------------+-------+-----------------+----+
|NULL                                                |NULL   |NULL             |1   |
|The Next Karate Kid                                 |Movie  |United States    |1   |
|To All the Boys: Always and Forever - The Afterparty|Movie  |June 12, 2021    |2   |
|What Would Sophia Loren Do?                         |Movie  |January 15, 2021 |3   |
|An Imperfect Murder                                 |Movie  |January 13, 2021 |4   |
|An Easy Girl                                        |Movie  |August 13, 2020  |5   |
|The Challenge                                       |TV Show|December 15, 2020|1   |
|Blood & Water                                       |TV Show|2021             |2   |
|Ganglands                                           |